In [2]:
import os

images = [os.path.join('data/images', fname) for fname in os.listdir('data/images')]

In [3]:
image_names = [os.path.splitext(os.path.basename(img))[0] for img in images]

In [ ]:
import json

with open('data/nocap_val_4500_captions.json', 'r') as f:
    captions_data = json.load(f)

: 

In [ ]:
grouped_results = []
for img in captions_data['images']:
    image_name = os.path.splitext(img['file_name'])[0]
    image_id = img['id']
    # Get all captions for this image_id
    image_captions = [
        {'captionid': cap['id'], 'caption': cap['caption']}
        for cap in captions_data['annotations'] if cap['image_id'] == image_id
    ]
    grouped_results.append({
        'image_name': image_name,
        'image_id': image_id,
        'captions': image_captions
    })

In [ ]:
grouped_results[:1]

In [ ]:
from PIL import Image

import matplotlib.pyplot as plt

img_path = 'data/validation/' + grouped_results[0]['image_name'] + '.jpg'
img = Image.open(img_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
import torch

# import vision transformer

from vit_pytorch.simple_vit_with_patch_dropout import SimpleViT
from vit_pytorch.extractor import Extractor

vit = SimpleViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    patch_dropout = 0.5  # https://arxiv.org/abs/2212.00794
)

vit = Extractor(vit, return_embeddings_only = True, detach = False)

In [ ]:
from coca_pytorch.coca_pytorch import CoCa

coca = CoCa(
    dim = 512,                     # model dimension
    img_encoder = vit,             # vision transformer - image encoder, returning image embeddings as (batch, seq, dim)
    image_dim = 1024,              # image embedding dimension, if not the same as model dimensions
    num_tokens = 20000,            # number of text tokens
    unimodal_depth = 6,            # depth of the unimodal transformer
    multimodal_depth = 6,          # depth of the multimodal transformer
    dim_head = 64,                 # dimension per attention head
    heads = 8,                     # number of attention heads
    caption_loss_weight = 1.,      # weight on the autoregressive caption loss
    contrastive_loss_weight = 1.,  # weight on the contrastive loss between image and text CLS embeddings
).cuda()

In [ ]:
import json, os, collections

ANN = 'data/nocap_val_4500_captions.json'
IMG_DIR = 'data/validation'  # where you downloaded images

with open(ANN, 'r') as f:
    ann = json.load(f)

# maps
id2file = {img['id']: img['file_name'] for img in ann['images']}
file2id = {v:k for k,v in id2file.items()}

# group refs by image_id
caps_by_id = collections.defaultdict(list)
for a in ann['annotations']:
    caps_by_id[a['image_id']].append(a['caption'])

num_images = len(ann['images'])
lens = [len(caps_by_id[i['id']]) for i in ann['images']]
print(f"#images: {num_images}, min refs: {min(lens)}, max refs: {max(lens)}, mean refs: {sum(lens)/len(lens):.2f}")

# sanity: list any images not having exactly 10 refs
bad = [(i['id'], id2file[i['id']], len(caps_by_id[i['id']])) for i in ann['images'] if len(caps_by_id[i['id']]) != 10]
print("non-10 reference counts:", len(bad))
bad[:5]


In [ ]:
import os
from PIL import Image
from tqdm import tqdm

def load_image(path):
    return Image.open(path).convert("RGB")

def generate_caption(model, pil_img):
    # TODO: YOUR decoding here (greedy/beam). Return a plain string.
    # Example placeholder:
    return "a placeholder caption for this image"

# Produce predictions for ALL images in the JSON file order
preds = []
for img in tqdm(ann['images']):
    image_id = img['id']
    img_path = os.path.join(IMG_DIR, img['file_name'])
    pil_img = load_image(img_path)

    caption = generate_caption(model=None, pil_img=pil_img)  # replace model=None with your CoCa
    preds.append({"image_id": image_id, "caption": caption})

len(preds), preds[:2]


In [ ]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import json, tempfile

# build a “results” json file from preds
tmp_results = tempfile.NamedTemporaryFile(suffix=".json", delete=False).name
with open(tmp_results, "w") as f:
    json.dump(preds, f)

coco = COCO(ANN)
cocoRes = coco.loadRes(tmp_results)

evaluator = COCOEvalCap(coco, cocoRes)
evaluator.evaluate()

# Print key metrics
for k, v in evaluator.eval.items():
    print(f"{k:8s}: {v:.4f}")


In [ ]:
import open_clip
import torch
from PIL import Image

''' 
('coca_ViT-B-32', 'laion2b_s13b_b90k'),
 ('coca_ViT-B-32', 'mscoco_finetuned_laion2b_s13b_b90k'),
 ('coca_ViT-L-14', 'laion2b_s13b_b90k'),
 ('coca_ViT-L-14', 'mscoco_finetuned_laion2b_s13b_b90k'),
 '''

model, _, transform = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2b_s13b_b90k"
)

im = Image.open("Coca/images/golden.jpeg").convert("RGB")
im = transform(im).unsqueeze(0)

with torch.no_grad(), torch.cuda.amp.autocast():
  generated = model.generate(im)

print(open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", ""))

1 a picture of one of the golden retreiver on the beach . one of the golden retreiver on the beach is looking at the camera
2 a brown dog is sitting on the beach
3 dog friendly beaches in sydney
4 a dog sitting on top of a sandy beach next to the ocean